In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [6]:
# a. LOAD PREPARED DATASET 2

dataset2_path = r"D:dataset/Object Detection(Ass6)"

img_height = 224
img_width = 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    dataset2_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    dataset2_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

num_classes = train_gen.num_classes
print("\nNumber of Classes Found:", num_classes)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:dataset\\Object Detection(Ass6)\\dataset2'

In [3]:
# b. LOAD PRE-TRAINED CNN (VGG16)
vgg = VGG16(
    weights=None,                
    include_top=False,
    input_shape=(224, 224, 3)
)

vgg.load_weights(
    r"dataset\Object Detection(Ass6)\vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
)

print("\nLoaded VGG16 Pretrained Weights Successfully!")


# Freeze all convolutional layers
for layer in vgg.layers:
    layer.trainable = False


Loaded VGG16 Pretrained Weights Successfully!


In [4]:
# c. ADD CUSTOM CLASSIFIER

x = Flatten()(vgg.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

NameError: name 'num_classes' is not defined

In [7]:
# d. Train classifier layers (with backbone frozen)

history_1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

NameError: name 'model' is not defined

In [8]:
# e. Fine-tune hyperparameters and unfreeze more layers

# Unfreeze last 4 layers of VGG16
for layer in vgg.layers[-4:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)

NameError: name 'model' is not defined

In [ ]:
# FINAL MODEL EVALUATION

loss, accuracy = model.evaluate(val_gen)
print(f"\n✅ Final Validation Accuracy: {accuracy*100:.2f}%")


# ---------------------------------------------------------
# TRAINING PLOTS
# ---------------------------------------------------------

def plot_results(h1, h2):
    acc = h1.history['accuracy'] + h2.history['accuracy']
    val_acc = h1.history['val_accuracy'] + h2.history['val_accuracy']
    loss = h1.history['loss'] + h2.history['loss']
    val_loss = h1.history['val_loss'] + h2.history['val_loss']

    epochs = range(1, len(acc)+1)

    plt.figure(figsize=(14,5))

    plt.subplot(1,2,1)
    plt.plot(epochs, acc, label="Train Accuracy")
    plt.plot(epochs, val_acc, label="Val Accuracy")
    plt.title("Accuracy")

    plt.subplot(1,2,2)
    plt.plot(epochs, loss, label="Train Loss")
    plt.plot(epochs, val_loss, label="Val Loss")
    plt.title("Loss")
    plt.show()

plot_results(history_1, history_2)